In [1]:
import pandas as pd
import numpy as np
import os
import pickle

# Generate training data using NY Times ingredient phrase tagger
from sklearn.model_selection import train_test_split

# Model libraries
from tagger_model import *

from IPython.core.debugger import set_trace

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Some default parameters
n_word_embedding_nodes=300
n_tag_embedding_nodes=150
n_RNN_nodes=400
n_dense_nodes=200

dataPath = '../data'

In [9]:
filename = 'ner_dataset.csv'
data = pd.read_csv(os.path.join(dataPath, filename), encoding="latin1")
data = data.fillna(method="ffill")

# Reformat data so that each sentence is put into a vector per row of a pandas dataframe
cleaned_dat = data.groupby('Sentence #', sort=False).apply(lambda x: pd.DataFrame(data = {'sents': [x.Word.tolist()], 
                                                                                       'tags': [x.Tag.tolist()]}))

In [10]:
train, test = train_test_split(cleaned_dat, test_size = .2)

In [11]:
# Create lexicon
lexicon = lexiconTransformer(words_min_freq=2, unknown_tag_token='OTHER')

lexicon.fit(train.sents, train.tags)

train['sent_indx'], train['tag_indx'] = lexicon.transform(train.sents, train.tags)

# Get length of longest sequence
max_seq_len = get_max_seq_len(train['sent_indx'])

#Add one to max length for offsetting sequence by 1
train_padded_words = pad_idx_seqs(train['sent_indx'], 
                                  max_seq_len + 1) 

train_padded_tags = pad_idx_seqs(train['tag_indx'],
                                 max_seq_len + 1)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
# Convert to one-hot vector encoding for y
# train_y = [to_categorical(i, num_classes=len(lexicon.tags_lexicon) + 1) for i in train_padded_tags]

In [13]:
mod_save_name = 'pos_model_crf_wordOnly'
crf_mod = True

In [14]:
ingredient_model = run_training_model(train_padded_words, train_padded_tags, 
                                      train_padded_tags, mod_save_name, lexicon, crf=crf_mod,
                                      print_summary=True, batch_size=256, epochs=10,
                                      n_word_embedding_nodes=n_word_embedding_nodes,
                                      n_tag_embedding_nodes=n_tag_embedding_nodes,
                                      n_RNN_nodes=n_RNN_nodes, 
                                      n_dense_nodes=n_dense_nodes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input_layer (InputLayer (None, 104)               0         
_________________________________________________________________
word_embedding_layer (Embedd (None, 104, 300)          5477400   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 104, 800)          1682400   
_________________________________________________________________
dense_layer (TimeDistributed (None, 104, 200)          160200    
_________________________________________________________________
output_layer (CRF)           (None, 104, 19)           4218      
Total params: 7,324,218
Trainable params: 7,324,218
Non-trainable params: 0
_________________________________________________________________
Train on 30693 samples, validate on 7674 samples
Epoch 1/10


InternalError: Blas GEMM launch failed : a.shape=(26368, 19), b.shape=(19, 19), m=26368, n=19, k=19
	 [[Node: loss/output_layer_loss/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/output_layer_loss/Reshape_3, loss/output_layer_loss/Reshape_4)]]
	 [[Node: metrics/acc/while_1/Identity/_272 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4753_metrics/acc/while_1/Identity", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^_cloopmetrics/acc/while_1/ExpandDims/dim/_145)]]

Caused by op 'loss/output_layer_loss/MatMul_1', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.

In [ ]:
test['sent_indx'], test['tag_indx'] = lexicon.transform(test.sents, test.tags)

In [ ]:
test_mod = create_test_model(mod_save_name, lexicon, crf=crf_mod, 
                             n_word_embedding_nodes=n_word_embedding_nodes,
                             n_tag_embedding_nodes=n_tag_embedding_nodes,
                             n_RNN_nodes=n_RNN_nodes, 
                             n_dense_nodes=n_dense_nodes)

In [ ]:
preds = predict_new_tag(test_mod, test[:50], lexicon)

In [ ]:
evaluate_model(preds, test[:50], print_sample=True)